In [1]:
# import lib
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_2012 = pd.read_csv('data/raw/2012_Data.csv', encoding='latin1', low_memory=False)
df_2013 = pd.read_csv('data/raw/2013_Data.csv', encoding='latin1', low_memory=False)

In [ ]:
# Concatenate the two DataFrames
merged_data = pd.concat([df_2012, df_2013], ignore_index=True)

# Display the first few rows of the merged data
merged_data.info()

# Data Cleaning

1. Handling Missing Values
- Identifying Missing Data: Determine where missing values occur and why (e.g., data entry errors, collection issues).
- Strategies for Handling Missing Data:
    - Removal: Drop rows or columns with excessive missing values if they don’t significantly affect the analysis.
    - Imputation: Replace missing values using techniques like mean, median, mode, or more sophisticated methods (e.g., regression or k-nearest neighbors).
2. Correcting Inconsistent Data
- Fix inconsistencies in data entries (e.g., different formats for the same value, such as "USA" and "United States").
- Standardize formats for dates, phone numbers, and other data types.
3. Removing Duplicates
- Identify and remove duplicate records, which can skew analysis results and lead to incorrect conclusions.
4. Handling Outliers
- Outliers can distort analysis, especially in statistical modeling. Identify them using visualization techniques (e.g., boxplots).
- Decide whether to remove, transform, or keep outliers based on the context and goals of the analysis.
5. Data Type Conversion
- Ensure that each column has the appropriate data type (e.g., numeric, categorical, datetime).
- Convert data types if needed (e.g., parsing date strings into datetime objects).
6. Normalization and Scaling
- Normalization: Rescale numerical data to fit within a particular range, often [0, 1].
- Standardization: Adjust the data to have a mean of zero and a standard deviation of one, which is important for certain machine learning algorithms.
7. Handling Categorical Variables
- Convert categorical variables to numeric form if required for analysis (e.g., one-hot encoding).
- Combine similar categories to reduce complexity (e.g., grouping rare categories together).
8. Feature Engineering
- Creating New Features: Generate new variables based on existing ones to capture additional insights.
- Dropping Irrelevant Features: Remove features that do not add value or are highly correlated, to avoid multicollinearity.
9. Text Cleaning (for Textual Data)
- For textual data, cleaning involves removing punctuation, converting to lowercase, removing stopwords, and stemming or lemmatizing words to bring them to their root form.

## 1. Handling Missing Values
- Identifying Missing Data: Determine where missing values occur and why (e.g., data entry errors, collection issues).
- Strategies for Handling Missing Data:
    - Removal: Drop rows or columns with excessive missing values if they don’t significantly affect the analysis.
    - Imputation: Replace missing values using techniques like mean, median, mode, or more sophisticated methods (e.g., regression or k-nearest neighbors).

In [ ]:
# check the length of data and rows.
merged_data.shape

In [ ]:
# indentifying the null values.
merged_data.isnull().sum()

The null values are shown in column 'item_source_class', which is 1,988,382, the same as the normal data length. This means this column does not use, so we decided to remove  it.

In [ ]:
# remove null values.
merged_data.drop('item_source_class', axis=1, inplace=True)

In [ ]:
# recheck null value after remove
merged_data.isnull().sum()

We expected that each row in each column would be unique, as each row represents a specific order item from a customer. If there are duplicate rows in any column, it could indicate an issue with data accuracy.

In [ ]:
# check the length of data and rows.
merged_data.shape

## 2. Correcting Inconsistent Data
- Fix inconsistencies in data entries (e.g., different formats for the same value, such as "USA" and "United States").
- Standardize formats for dates, phone numbers, and other data types.

First, we want to check whether the accounting date, fiscal year, and calendar year are consistent. However, the accounting date is not in the same standardised format as the others, so we will convert it to match the same format before checking consistency.

In [ ]:
# Convert 'accounting_date' to datetime format
merged_data['accounting_date'] = pd.to_datetime(merged_data['accounting_date'], format='%Y%m%d')

# Create separate columns for year, month, and day from the datetime object
merged_data['accounting_year'] = merged_data['accounting_date'].dt.year
merged_data['accounting_month'] = merged_data['accounting_date'].dt.month
merged_data['accounting_day'] = merged_data['accounting_date'].dt.day

# Display the first few rows to verify the results
merged_data.head()

Now, there are new columns: 'accounting_year', 'accounting_month', and 'accounting_day'. We will use these columns to check for consistency.

In [ ]:
# Sort and print unique values in ascending order for calendar year, month, and day
print("Calendar Year:", sorted(merged_data['calendar_year'].unique()))
print("Calendar Month:", sorted(merged_data['calendar_month'].unique()))
print("Calendar Day:", sorted(merged_data['calendar_day'].unique()))

print("\n")

# Sort and print unique values in ascending order for accounting year, month, and day
print("Accounting Year:", sorted(merged_data['accounting_year'].unique()))
print("Accounting Month:", sorted(merged_data['accounting_month'].unique()))
print("Accounting Day:", sorted(merged_data['accounting_day'].unique()))


The results show that the calendar column and the accounting column are consistent. Next, we will check the consistency between the fiscal column and the calendar column.

In [ ]:
# Sort and print unique values in ascending order for calendar year and month
print("Calendar Year:", sorted(merged_data['calendar_year'].unique()))
print("Calendar Month:", sorted(merged_data['calendar_month'].unique()))

print("\n")

# Sort and print unique values in ascending order for fiscal year and month
print("Fisical Year:", sorted(merged_data['fiscal_year'].unique()))
print("Fisical Month:", sorted(merged_data['fiscal_month'].unique()))


We observed that the fiscal year and month had unique patterns compared to others. Therefore, we examined the number of months within each fiscal year and identified the specific months for each year.

In [ ]:
# Group by 'fiscal_year' and list all unique 'fiscal_month' for each year
fiscal_months_per_year = merged_data.groupby('fiscal_year')['fiscal_month'].apply(lambda x: sorted(x.unique()))

# Display the result
print(fiscal_months_per_year)

Upon review, we found that the fiscal year follows the financial calendar in Australia. This means that the fiscal year 2012 includes July to December 2012 and January to June 2013, while the fiscal year 2013 includes July to December 2013 and January to June 2014. In summary, the fiscal year is the same as the calendar and accounting year.

In the next step, we decided to fix inconsistencies in data entries for the following columns: 'order_type_code', 'abc_class_volume', 'abc_class_code', 'warehouse_code', 'environment_group_code', 'business_area_code', 'customer_district_code', and 'technology_group_code'.

- **order_type_code**

In [ ]:
# Check unique order type
print(merged_data['order_type_code'].unique())

It shows that 'PME' does not have any list in our document, so we check how many rows show 'PME'

In [ ]:
# Filter the DataFrame for rows where 'order_type_code' is 'PME'
pme_rows = merged_data[merged_data['order_type_code'] == 'PME']

# Count the number of rows
pme_count = len(pme_rows)

# Display the count
print(f"Number of rows with 'order_type_code' as 'PME': {pme_count}")

It resulted in 243 rows, so we decided to drop them.

In [ ]:
# Drop rows where 'order_type_code' is 'PME'
merged_data = merged_data[merged_data['order_type_code'] != 'PME']

# Display the first few rows to confirm
merged_data.shape

- **abc_class_volume**

In [ ]:
# Check unique class volume
print(merged_data['abc_class_volume'].unique())

The results show consistency and a uniform format.

- **abc_class_code**

In [ ]:
# Check unique class code
print(merged_data['abc_class_code'].unique())

The results show consistency and a uniform format.

- **warehouse_code**

In [ ]:
# Check unique warehouse code
print(merged_data['warehouse_code'].unique())

The results show that there are many warehouse codes, so we need to double-check them against our reference list to identify any discrepancies.

In [ ]:
# List of valid warehouse codes from our reference
valid_warehouse_codes = [
    'Q1', 'V0', 'S0', 'T0', 'Q0', 'N0', 'KN0', 'W0', 'S1', 'Unk', 'GS0', 'GN0', 
    'CN0', 'CQ0', 'CV0', 'GW0', 'CS0', 'CW0', 'GQ0', 'AS0', 'CT0', 'CS1', 'CN1', 
    'CZ0', 'LW0', 'LQ0', 'LS1', 'LV0', 'LN9', 'LS0', 'FWE', 'EN0', 'FW2', 'JT0', 
    'FA1', 'FWA', 'FA2', '1N0', '1S0', '5N2', '5V0', '5S0', '5W0', '1V0', '1Q0', 
    '1N1', '1W0', '1Q1', '5Q0', '5T0', '5S1', '5N1', '1T0', '1S1', '5Q1']

# Filter the DataFrame to get only the invalid warehouse codes
invalid_warehouse_data = merged_data[~merged_data['warehouse_code'].isin(valid_warehouse_codes)]

# Group by the invalid warehouse codes and count their occurrences
invalid_code_counts = invalid_warehouse_data['warehouse_code'].value_counts()

# Display the grouped counts of invalid codes
print("Counts of invalid warehouse codes:")
print(invalid_code_counts)
len(invalid_code_counts)

The results show some invalid entries compared to our reference list. However, the codes 'V0', 'N0', 'Q0', 'W0', 'S0', 'Q1', 'T0', and 'S1' are present in our list but contain extra spaces. Therefore, we need to correct their format by removing the spaces.

In [ ]:
# Define a dictionary to map codes with spaces to the cleaned versions
replacement_map = {
    'V0 ': 'V0',
    'N0 ': 'N0',
    'Q0 ': 'Q0',
    'W0 ': 'W0',
    'S0 ': 'S0',
    'Q1 ': 'Q1',
    'T0 ': 'T0',
    'S1 ': 'S1'}

# Replace values in the 'warehouse_code' column based on the mapping
merged_data['warehouse_code'] = merged_data['warehouse_code'].replace(replacement_map)

# Verify the replacements
print("Unique warehouse codes after replacement:")
print(merged_data['warehouse_code'].unique())

In [ ]:
# After cleaning, check for invalid warehouse codes again
invalid_warehouse_data = merged_data[~merged_data['warehouse_code'].isin(valid_warehouse_codes)]

# Count occurrences of each invalid warehouse code
invalid_code_counts = invalid_warehouse_data['warehouse_code'].value_counts()

# Display the counts of invalid warehouse codes
print("Counts of invalid warehouse codes after cleaning:")
print(invalid_code_counts)
len(invalid_warehouse_data)

After rechecking, we found that the 'warehouse_code' has data invalid around 215 rows, So we decided to delete it.

In [ ]:
# List of invalid warehouse codes to drop
invalid_codes = ['1T1', 'BB1', '1N2', '1N3']

# Drop rows where 'warehouse_code' is one of the invalid codes
merged_data = merged_data[~merged_data['warehouse_code'].isin(invalid_codes)]

# Display the shape to confirm rows have been dropped
print(merged_data.shape)

- **environment_group_code**

In [ ]:
# check unique environment column
print(merged_data['environment_group_code'].unique())

The results show that the environment column codes contain extra spaces, so I will replace them int correct format.

In [ ]:
# Define a dictionary to map codes with spaces to the cleaned versions
replacement_environ = {
    'S                             ': 'S',
    'P                             ': 'P',
    'D                             ': 'D',
    'Z                             ': 'Z',
    'C                             ': 'C',
    'M                             ': 'M',
    'R                             ': 'R',
    'I                             ': 'I',
    'NA                            ': 'NA'}

# Replace values in the 'environment_group_code' column based on the mapping
merged_data['environment_group_code'] = merged_data['environment_group_code'].replace(replacement_environ)
unique_warehouse_code = merged_data['environment_group_code'].unique()
print(unique_warehouse_code)

Moreover, we checked the code against our environment group list, and no invalid entries were found.

- **business_area_code**

In [ ]:
# check the unique business code
print(merged_data['business_area_code'].unique())

The results show that there are many business codes, so we need to double-check them against our reference list to identify any discrepancies.

In [ ]:
# List of valid business codes from your reference
business_area_codes = {
    'LMP', 'FLD', 'OTH', 'SUR', 'COM', 'DLT', 'TRO', 'URB', 'HLB', 'SAE', 'RWY', 
    'LCP', 'PEN', 'EXL', 'TAL', '945', '950', '980', '920', '960', '910', '930', 
    '999', '970', '940', '985', 'IAE', 'IAI'}

# Filter the DataFrame to get only the invalid business codes
invalid_business_data = merged_data[~merged_data['business_area_code'].isin(business_area_codes)]

# Group by the invalid business codes and count their occurrences
invalid_business_counts = invalid_business_data['business_area_code'].value_counts()

# Display the grouped counts of invalid codes
print("Counts of invalid business codes:")
print(invalid_business_counts)

The results show some invalid entries compared to our reference list. However, some of the codes are present in our list but contain extra spaces. Therefore, we need to correct their format by removing the spaces.

In [ ]:
# Define a dictionary to map codes with spaces to the cleaned versions
replacement_business_code = {
    'LMP                           ': 'LMP',
    'FLD                           ': 'FLD',
    'OTH                           ': 'OTH',
    'SUR                           ': 'SUR',
    'COM                           ': 'COM',
    'DLT                           ': 'DLT',
    'TRO                           ': 'TRO',
    'URB                           ': 'URB',
    'HLB                           ': 'HLB',
    'SAE                           ': 'SAE',
    'RWY                           ': 'RWY',
    'LCP                           ': 'LCP',
    'PEN                           ': 'PEN',
    'EXL                           ': 'EXL',
    'TAL                           ': 'TAL',
    'IAE                           ': 'IAE',
    'IAI                           ': 'IAI'}

# Replace values in the 'business_area_code' column based on the mapping
merged_data['business_area_code'] = merged_data['business_area_code'].replace(replacement_business_code)

# Verify the replacements
print("Unique business codes after replacement:")
print(merged_data['business_area_code'].unique())

In [ ]:
# After cleaning, check for invalid business codes again
invalid_business_data = merged_data[~merged_data['business_area_code'].isin(business_area_codes)]

# Count occurrences of each invalid business code
invalid_business_counts = invalid_business_data['business_area_code'].value_counts()

# Display the counts of invalid business codes
print("Counts of invalid business codes:")
print(invalid_business_counts)

After rechecking, we found that the 'business_area_code' has not data invalid.

- **customer_district_code**

In [ ]:
# check the unique customer district column
print(merged_data['customer_district_code'].unique())

The results show that there are many customer codes, so we need to double-check them against our reference list to identify any discrepancies.

In [ ]:
# List of valid customer district codes from your reference
customer_district_code = {'410', '300', '500', '310', '400', '200', '210', '720', '710', '600',
                          '510', '530', '535', '540', '520', '545'}

# Filter the DataFrame to get only the invalid customer codes
invalid_customer_data = merged_data[~merged_data['customer_district_code'].astype(str).isin(customer_district_code)]

# Group by the invalid customer codes and count their occurrences
invalid_customer_counts = invalid_customer_data['customer_district_code'].value_counts()

# Display the grouped counts of invalid codes
print("Counts of invalid business codes:")
print(invalid_customer_counts)

After rechecking, we found that the 'customer_district_code' has data invalid around 6 rows, So we decided to delete it.

In [ ]:
# List of invalid customer codes to drop
invalid_customer = ['100']

# Drop rows where 'customer_district_code' is one of the invalid codes
merged_data = merged_data[~merged_data['customer_district_code'].astype(str).isin(invalid_customer)]

# Display the shape to confirm rows have been dropped
print(merged_data.shape)

In [ ]:
# After cleaning, check for invalid customer codes again
invalid_customer_data = merged_data[~merged_data['customer_district_code'].astype(str).isin(customer_district_code)]

# Count occurrences of each invalid customer code
invalid_customer_counts = invalid_customer_data['customer_district_code'].value_counts()

# Display the counts of invalid customer codes
print("Counts of invalid business codes:")
print(invalid_customer_counts)

- **technology_group_code**

In [ ]:
# check the unique technology code
print(merged_data['technology_group_code'].unique())

The results show that there are many technology codes, so we need to double-check them against our reference list to identify any discrepancies.

In [ ]:
# List of valid technology codes from your reference
technology_group_codes = [
    'SYLV', 'NA', 'PIER', '96', '219', '92', '214', '380', '102', '123', '110', 
    '98', '213', '580', '108', '999', '198', 'SCHR', '82', '140', '540', 'AUST', 
    '480', '760', '460', '600', '520', '998', '570', '280', '330', '220', '415', 
    '340', '225', 'CSE', '565', '410', '560', '800', '680', '320', '550', '400', 
    '160', '440', '420', '500', '555', '360', '290', '780', '545', '181', '240', 
    '640', '820', '595', '300', '830', '70', '850', '86', '720', '740', '310', 
    'FGDFT', '350', '206', '64', 'PNZ', '63', 'CROM', '100', '78', '207', '208', 
    '76', '211', '205', '85', '90', '68', '130', 'GLG', 'INLIT', '72', '61', 
    '880', '217', '215', '80', '210', 'INZ', '250', '118', '66']

# Filter the DataFrame to get only the invalid technology codes
invalid_technology_data = merged_data[~merged_data['technology_group_code'].isin(technology_group_codes)]

# Group by the invalid technology codes and count their occurrences
invalid_technology_counts = invalid_technology_data['technology_group_code'].value_counts()

# Display the grouped counts of invalid codes
print("Counts of invalid technology codes:")
print(invalid_technology_counts)
len(invalid_technology_counts)

The results show some invalid entries compared to our reference list. However, some of the codes are present in our list but contain extra spaces. Therefore, we need to correct their format by removing the spaces.

In [ ]:
# Define a dictionary to map codes with spaces to the cleaned versions
replacement_technology_code = {
    'SYLV                                    ': 'SYLV',
    'NA                                      ': 'NA',
    'PIER                                    ': 'PIER',
    'SCHR                                    ': 'SCHR',
    'AUST                                    ': 'AUST',
    'CSE                                     ': 'CSE',
    'FGDFT                                   ': 'FGDFT',
    'PNZ                                     ': 'PNZ',
    'CROM                                    ': 'CROM',
    'GLG                                     ': 'GLG',
    'INLIT                                   ': 'INLIT',
    'INZ                                     ': 'INZ',
    'PHANT                                   ': 'PHANT',
    'DIGIN                                   ': 'DIGIN'}

# Replace values in the 'technology_group_code' column based on the mapping
merged_data['technology_group_code'] = merged_data['technology_group_code'].replace(replacement_technology_code)

# Verify the replacements
print("Unique technology codes after replacement:")
print(merged_data['technology_group_code'].unique())

In [ ]:
# After cleaning, check for invalid technology codes again
invalid_technology_data = merged_data[~merged_data['technology_group_code'].isin(technology_group_codes)]

# Count occurrences of each invalid technology code
invalid_technology_counts = invalid_technology_data['technology_group_code'].value_counts()

# Display the counts of invalid technology codes
print("Counts of invalid technology codes:")
print(invalid_technology_counts)
len(invalid_technology_data)

After rechecking, we found that the 'technology_group_code' has data invalid around 210 rows, So we decided to delete it.

In [ ]:
# List of invalid technology codes to drop
invalid_technology = ['128', 'DIGIN', 'PHANT', '88', '114', '112']

# Drop rows where 'technology_group_code' is one of the invalid codes
merged_data = merged_data[~merged_data['technology_group_code'].isin(invalid_technology)]

# Display the shape to confirm rows have been dropped
print(merged_data.shape)

- currency

In [ ]:
# check the unique currency
print(merged_data['currency'].unique())

In [ ]:
# Count the occurrences of each unique currency
print(merged_data['currency'].value_counts())

Following an analysis of the currency counts, we decided to remove entries with blank spaces and standardize 'AUS' to 'AUD'.

In [ ]:
# Replace 'AUS' with 'AUD'
merged_data['currency'] = merged_data['currency'].replace('AUS', 'AUD')

# Drop rows where 'currency' is blank or contains only spaces
merged_data = merged_data[merged_data['currency'].str.strip() != '']

# Verify the changes
print(merged_data['currency'].value_counts())

## 3. Removing Duplicates
- Identify and remove duplicate records, which can skew analysis results and lead to incorrect conclusions.

In [ ]:
# Check for duplicate rows based on all columns
duplicate_rows = merged_data[merged_data.duplicated(keep='first')]

# Display the duplicate rows
print("Duplicate rows based on all columns:")
display(duplicate_rows)

After identifying the duplicate records, it shows the duplicate records 8,209 rows, which we will remove.

In [ ]:
# Remove duplicate rows based on all columns
cleaned_data = merged_data.drop_duplicates()

# Display the shape to confirm duplicates were removed
print("Data after removing duplicates:", cleaned_data.shape)

## 4. Handling Outliers
- Outliers can distort analysis, especially in statistical modeling. Identify them using visualization techniques (e.g., boxplots).
- Decide whether to remove, transform, or keep outliers based on the context and goals of the analysis.

- Focusing on value-related columns, value_sales, value_cost, value_quantity is critical because these directly impact financial performance and operational decisions.
- Based on the output of value_price_adjustment, it seems that this column primarily contains binary values (0 and 1), with an overwhelming majority being 0. This suggests that value_price_adjustment is likely a categorical or indicator variable rather than a continuous one.

In [ ]:
print(merged_data['value_price_adjustment'].value_counts())

In [ ]:
import matplotlib.pyplot as plt

# Set up the figure with a grid of 1x3 subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot the box plot for 'value_sales' in the first subplot
merged_data['value_sales'].plot(kind='box', ax=axes[0])
axes[0].set_title('Boxplot of value_sales')

# Plot the box plot for 'value_cost' in the second subplot
merged_data['value_cost'].plot(kind='box', ax=axes[1])
axes[1].set_title('Boxplot of value_cost')

# Plot the box plot for 'value_quantity' in the third subplot
merged_data['value_quantity'].plot(kind='box', ax=axes[2])
axes[2].set_title('Boxplot of value_quantity')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()


In [ ]:
# For value_sales
Q1_sales = merged_data['value_sales'].quantile(0.25)
Q3_sales = merged_data['value_sales'].quantile(0.75)
IQR_sales = Q3_sales - Q1_sales
lower_bound_sales = Q1_sales - 1.5 * IQR_sales
upper_bound_sales = Q3_sales + 1.5 * IQR_sales
print("Bounds for value_sales:", lower_bound_sales, "-", upper_bound_sales)

# For value_cost
Q1_cost = merged_data['value_cost'].quantile(0.25)
Q3_cost = merged_data['value_cost'].quantile(0.75)
IQR_cost = Q3_cost - Q1_cost
lower_bound_cost = Q1_cost - 1.5 * IQR_cost
upper_bound_cost = Q3_cost + 1.5 * IQR_cost
print("Bounds for value_cost:", lower_bound_cost, "-", upper_bound_cost)

# For value_quantity
Q1_quantity = merged_data['value_quantity'].quantile(0.25)
Q3_quantity = merged_data['value_quantity'].quantile(0.75)
IQR_quantity = Q3_quantity - Q1_quantity
lower_bound_quantity = Q1_quantity - 1.5 * IQR_quantity
upper_bound_quantity = Q3_quantity + 1.5 * IQR_quantity
print("Bounds for value_quantity:", lower_bound_quantity, "-", upper_bound_quantity)


In [ ]:
# Filter outliers for each column
sales_outliers = merged_data[(merged_data['value_sales'] < lower_bound_sales) | (merged_data['value_sales'] > upper_bound_sales)]
cost_outliers = merged_data[(merged_data['value_cost'] < lower_bound_cost) | (merged_data['value_cost'] > upper_bound_cost)]
quantity_outliers = merged_data[(merged_data['value_quantity'] < lower_bound_quantity) | (merged_data['value_quantity'] > upper_bound_quantity)]

# Display only the outliers for each column
print("Sales Outliers:\n", sales_outliers[['value_sales']])
print("Cost Outliers:\n", cost_outliers[['value_cost']])
print("Quantity Outliers:\n", quantity_outliers[['value_quantity']])


Setting thresholds 500,000 for value_sales, 600,000 for value_cost, and 50,000 for value_quantity helps us identify unusually large or small transactions based on what could be considered reasonable or typical in a business context.
By defining these limits, you can filter out transactions that might represent rare, high-impact events or possible data errors.

Using 500,000 for value_sales, 600,000 for value_cost, and 50,000 for value_quantity is a starting point based on:

- Visual Analysis of Box Plots: The plots indicated values around these thresholds as extremes.
- Business Logic: These numbers reflect what could be a reasonable upper limit for most cases, while anything beyond might be rare and worth investigating.

In [ ]:
# Hypothetical business thresholds (modify based on real business knowledge)
sales_threshold = 500000
cost_threshold = 600000
quantity_threshold = 50000


# Filter based on business-defined thresholds
business_sales_outliers = merged_data[(merged_data['value_sales'] > sales_threshold) | (merged_data['value_sales'] < -sales_threshold)]
business_cost_outliers = merged_data[(merged_data['value_cost'] > cost_threshold) | (merged_data['value_cost'] < -cost_threshold)]
business_quantity_outliers = merged_data[(merged_data['value_quantity'] > quantity_threshold) | (merged_data['value_quantity'] < -quantity_threshold)]

# Display only the outlier values for each column

print("Business Sales Outliers:\n", business_sales_outliers[['value_sales']])
print("Business Cost Outliers:\n", business_cost_outliers[['value_cost']])
print("Business Quantity Outliers:\n", business_quantity_outliers[['value_quantity']])


- Remove extreme outliers based on business-defined limits.
- Focus the analysis on values that fall within a "reasonable" range, reducing the influence of extremely high or low values that might skew results.

In [ ]:
merged_data = merged_data[
    (merged_data['value_sales'] <= sales_threshold) & (merged_data['value_sales'] >= -sales_threshold) &
    (merged_data['value_cost'] <= cost_threshold) & (merged_data['value_cost'] >= -cost_threshold) &
    (merged_data['value_quantity'] <= quantity_threshold) & (merged_data['value_quantity'] >= -quantity_threshold)
]

- Boxplot of value_sales, value_cost, value_quantity after remove extreme outliers

In [ ]:
# Set up the figure with a grid of 1x3 subplots
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot the box plot for 'value_sales' in the first subplot
merged_data['value_sales'].plot(kind='box', ax=axes[0])
axes[0].set_title('Boxplot of value_sales')

# Plot the box plot for 'value_cost' in the second subplot
merged_data['value_cost'].plot(kind='box', ax=axes[1])
axes[1].set_title('Boxplot of value_cost')

# Plot the box plot for 'value_quantity' in the third subplot
merged_data['value_quantity'].plot(kind='box', ax=axes[2])
axes[2].set_title('Boxplot of value_quantity')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()


In [ ]:
# Plot histograms for each variable
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Histogram for value_sales
axes[0].hist(merged_data['value_sales'], bins=30, edgecolor='black')
axes[0].set_title('Histogram of value_sales')
axes[0].set_xlabel('value_sales')
axes[0].set_ylabel('Frequency')

# Histogram for value_cost
axes[1].hist(merged_data['value_cost'], bins=30, edgecolor='black')
axes[1].set_title('Histogram of value_cost')
axes[1].set_xlabel('value_cost')
axes[1].set_ylabel('Frequency')

# Histogram for value_quantity
axes[2].hist(merged_data['value_quantity'], bins=30, edgecolor='black')
axes[2].set_title('Histogram of value_quantity')
axes[2].set_xlabel('value_quantity')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

1. Highly Skewed Data:
The data for each of these columns is highly skewed, meaning there are many low or zero values and only a few high values.
This often occurs when there is a large number of small transactions (or even no transactions in some cases), and only a few large transactions.
2. Sparse Distribution:
The narrow and high central bars suggest that the vast majority of values are very close to zero, with few values spreading out across the range.
This could indicate that only a small percentage of transactions have high value_sales, value_cost, or value_quantity.
3. Potential Need for Transformation:
Because of the skewness, the data may benefit from a log transformation or another method to spread the values more evenly, especially if you plan to perform analyses or modeling that assume normality.
4. Zero or Minimal Transactions:
If there are many transactions with zero or minimal values in these columns, it might be useful to analyze these zero-value transactions separately. This could be a sign of canceled orders, unpaid invoices, or other business cases where no significant sales, cost, or quantity were recorded.

- Count zero values in each column

In [ ]:
# Count zero values in each column
zero_value_sales = (merged_data['value_sales'] == 0).sum()
zero_value_cost = (merged_data['value_cost'] == 0).sum()
zero_value_quantity = (merged_data['value_quantity'] == 0).sum()

# Display the counts
print("Number of zero values in value_sales:", zero_value_sales)
print("Number of zero values in value_cost:", zero_value_cost)
print("Number of zero values in value_quantity:", zero_value_quantity)


## 5. Data Type Conversion
- Ensure that each column has the appropriate data type (e.g., numeric, categorical, datetime).
- Convert data types if needed (e.g., parsing date strings into datetime objects).

In [ ]:
# Check data types of each column
print(merged_data.dtypes)

- Datetime conversion improves flexibility for time-based analysis, enables accurate date calculations, and enhances data filtering.

In [ ]:
merged_data['invoice_date'] = pd.to_datetime(merged_data['invoice_date'], format='%Y%m%d', errors='coerce')
merged_data['order_date'] = pd.to_datetime(merged_data['order_date'], format='%Y%m%d', errors='coerce')

- Categorical conversion reduces memory usage, speeds up processing, aids in machine learning, and maintains data integrity

In [ ]:
categorical_columns = [
    'customer_code', 'item_code', 'business_area_code', 'item_group_code', 'item_class_code',
    'bonus_group_code', 'environment_group_code', 'technology_group_code', 'commission_group_code',
    'reporting_classification', 'light_source', 'warehouse_code', 'abc_class_code', 'business_chain_l1_code',
    'business_chain_l1_name', 'contact_method_code', 'salesperson_code', 'order_type_code', 'market_segment', 
    'currency', 'customer_order_number'
]
merged_data[categorical_columns] = merged_data[categorical_columns].astype('category')

In [ ]:
print(merged_data.dtypes)

## 6. Normalization and Scaling
- Normalization: Rescale numerical data to fit within a particular range, often [0, 1].
- Standardization: Adjust the data to have a mean of zero and a standard deviation of one, which is important for certain machine learning 

## 7. Handling Categorical Variables
- Convert categorical variables to numeric form if required for analysis (e.g., one-hot encoding).
- Combine similar categories to reduce complexity (e.g., grouping rare categories together).

## 8. Feature Engineering
- Creating New Features: Generate new variables based on existing ones to capture additional insights.
- Dropping Irrelevant Features: Remove features that do not add value or are highly correlated, to avoid multicollinearity.

## 9. Text Cleaning (for Textual Data)
- For textual data, cleaning involves removing punctuation, converting to lowercase, removing stopwords, and stemming or lemmatizing words to bring them to their root form.